# Danish Sentence similarity

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# import os
# os.chdir('D:/DataScienceWorkSpace/danish_sentence_similarty/src')

In [3]:
# data fie path in local
data_file = '../data/sentences.csv'

In [4]:
# read the sentences in a dataframe
sentence_df = pd.read_csv(data_file)
sentence_df.head()

,id,text
0,10-12-176,"Vanddamp er en usynlig gas, der forekommer i s..."
1,10-13-182,Er der nogen herinde der har erfaring med at k...
2,10-14-29,Ved ikke lige hvordan disse er i størrelsen?
3,10-16-39,Dog kan jeg godt lide pang farver;)
4,10-17-297,Pengene bliver dog ofte først udbetalt efter 5...


In [5]:
# target sentence for finding similar senences
target_sentence = sentence_df.loc[sentence_df['id']=='7-21-440','text']
target_sentence.values[0]

'Ifølge Dansk Kennelklub angriber muskelhunde dyr og mennesker cirka hver 14. dag.'

English translation for this sentence - __According to the Danish Kennel Club, muscular dogs attack animals and humans approximately every 14 days.__

## Option 1: Toenization > cleaning > Count vectorizer / TF-IDF vectorizer > Similarity

In [14]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.snowball import DanishStemmer


from cleantext import clean

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [9]:
# tokenization function with Danish language Stemmer
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(DanishStemmer().stem(item))
    return stems

# clean text by lowering abd removing email, number, url, currency, punctuation, 
def normalize_text(text):
    return clean(text,
     no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=True,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=True,                # replace all digits with a special token
    no_currency_symbols=True,      # replace all currency symbols with a special token
    no_punct=True,
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="",
    replace_with_email="",
    replace_with_phone_number="",
    replace_with_number="",
    replace_with_currency_symbol="",
#     replace_with_punct=" ~PUNCT~ ",          # instead of removing punctuations you may replace them
#     replace_with_url=" ~URL~ ",
#     replace_with_email=" ~EMAIL~ ",
#     replace_with_phone_number=" ~PHONE~ ",
#     replace_with_number=" ~NUMBER~ ",
#     replace_with_currency_symbol=" ~CUR~ ",
    lang="de") # german is used as assumed that german would be closeer to Danish that english

# sample output for the target text
tokenize(normalize_text(target_sentence.values[0]))

['iflg',
 'dansk',
 'kennelklub',
 'angrib',
 'muskelhund',
 'dyr',
 'og',
 'mennesk',
 'cirka',
 'hver',
 'dag']

In [10]:
# create a replica of the sentence dataframe for processing
sentence_df_cp_op1 = sentence_df.copy()
sentence_df_cp_op1 = sentence_df_cp_op1.text.apply(normalize_text)
sentence_df_cp_op1

0       vanddamp er en usynlig gas der forekommer i st...
1       er der nogen herinde der har erfaring med at k...
2              ved ikke lige hvordan disse er i strrelsen
3                       dog kan jeg godt lide pang farver
4       pengene bliver dog ofte frst udbetalt efter da...
                              ...                        
4995    i dag i idrt skulle vi sa have bip test hvor j...
4996    p men tnkt nu hvis de bragte mere fra danskspr...
4997                             kathani hejsa allesammen
4998    weeeeeee jeg har mega optur pa de var begge mi...
4999    archon der blev jo ikke sagt noget om hvem der...
Name: text, Length: 5000, dtype: object

### Cosine similarity on count vectorizer

In [111]:
# fit count vectorizer with Danish stop words
# use the custom tokenizer with Danish Stemmer
count_vect = CountVectorizer(tokenizer=tokenize, stop_words=stopwords.words('danish'))
# fit on cleaned text
count_vect_matrix = count_vect.fit_transform(sentence_df_cp_op1)
# vocab sze 13574
count_vect_matrix.shape

(5000, 13574)

In [112]:
# find cosine similarity for the count vector matrix
cosine_matrix_cv = cosine_similarity(count_vect_matrix)

In [113]:
# convert to Data Frame
cosine_df_cv = pd.DataFrame(cosine_matrix_cv, columns = sentence_df['id'], index = sentence_df['id'])

In [114]:
# select the row for target sentence
target_cosine_array = cosine_df_cv.loc['7-21-440',:]
# sort descending
target_cosine_array.sort_values(ascending = False)[:10]

id
7-21-440      1.000000
47-2-28       0.316228
51-33-2622    0.258199
36-0-1231     0.239046
42-26-238     0.239046
6-37-308      0.223607
45-86-368     0.223607
38-8-2425     0.223607
51-89-2877    0.223607
20-10-28      0.223607
Name: 7-21-440, dtype: float64

In [115]:
# gather indices for top 10 similar sentences
top_10_similarity = target_cosine_array.sort_values(ascending = False)[:10].index.tolist()

In [123]:
sentence_df.loc[sentence_df.id == top_10_similarity[1],'text'].values[0]

'I dag'

In [122]:
sentence_df.loc[sentence_df.id == top_10_similarity[2],'text'].values[0]

'Hver dag i december op til juleaften sendes et afsnit.'

In [118]:
sentence_df.loc[sentence_df.id == top_10_similarity[3],'text'].values[0]

'Det er der min arbejdsinteresse ligger, og det er noget, som jeg kan bruge hver dag.'

In [119]:
sentence_df.loc[sentence_df.id == top_10_similarity[4],'text'].values[0]

'Danskerne er blandt de europæere, som hver for sig producerer mest affald.'

In [120]:
sentence_df.loc[sentence_df.id == top_10_similarity[5],'text'].values[0]

'Hovedårsagen til, at Katrina blev så dyr, at skaderne blev så omfangsrige, er ganske enkelt, at der i dag bor langt flere mennesker i kystområderne end tidligere.'

### Cosine similarity on TF-IDF vectorizer

In [127]:
# fit tf-idf vectorizer with Danish stop words
# use the custom tokenizer with Danish Stemmer
tfidf_vect = TfidfVectorizer(tokenizer=tokenize, stop_words=stopwords.words('danish'))
# fit on cleaned text
tfidf_vect_matrix = tfidf_vect.fit_transform(sentence_df_cp_op1)
tfidf_vect_matrix.shape

D:\Anaconda\envs\danish_nlp\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['all', 'and', 'bliv', 'dis', 'eft', 'ell', 'hav', 'havd', 'hend', 'ikk', 'kun', 'mang', 'meg', 'nog', 'nogl', 'skul', 'und', 'vær'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(5000, 13574)

In [128]:
# pairwise cosine similarity for tf-idf vector matrix
cosine_matrix_ti = cosine_similarity(tfidf_vect_matrix)

In [129]:
# convert to DF
cosine_df_ti = pd.DataFrame(cosine_matrix_ti, columns = sentence_df['id'], index = sentence_df['id'])

In [130]:
# select row for target sentence
target_cosine_array = cosine_df_ti.loc['7-21-440',:]
# sort descending
target_cosine_array.sort_values(ascending = False)[:10]

id
7-21-440      1.000000
47-2-28       0.216134
38-20-1059    0.180219
45-86-368     0.179803
36-0-1231     0.174177
38-78-1469    0.167071
34-2-723      0.166653
6-37-308      0.166108
40-81-2158    0.154190
51-33-2622    0.152718
Name: 7-21-440, dtype: float64

In [131]:
# Indices for top 10 similr sentences
top_10_similarity = target_cosine_array.sort_values(ascending = False)[:10].index.tolist()

In [132]:
sentence_df.loc[sentence_df.id == top_10_similarity[1],'text'].values[0]

'I dag'

In [133]:
sentence_df.loc[sentence_df.id == top_10_similarity[2],'text'].values[0]

'Jeg ved, at niveauet er højt blandt alle angriberne i Serie A og kræver en toppræstation hver eneste gang.'

In [134]:
sentence_df.loc[sentence_df.id == top_10_similarity[3],'text'].values[0]

'Hvad er dyrets yndlingsfoder?'

In [135]:
sentence_df.loc[sentence_df.id == top_10_similarity[4],'text'].values[0]

'Det er der min arbejdsinteresse ligger, og det er noget, som jeg kan bruge hver dag.'

In [136]:
sentence_df.loc[sentence_df.id == top_10_similarity[5],'text'].values[0]

'Men det fik ikke den farlige angriber til at skåne City.'

## Option 2: spaCy word embedding vector for sentence similarity

In [1]:
import spacy

In [2]:
# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("da_core_news_md")

In [15]:
# create replica for cleaned text df from last section
sentence_df_cp_op2 = sentence_df_cp_op1.copy()
sentence_df_cp_op2 = pd.DataFrame(sentence_df_cp_op2.values, columns = ['text'], index = sentence_df['id'])
sentence_df_cp_op2.head() 

,text
id,
10-12-176,vanddamp er en usynlig gas der forekommer i st...
10-13-182,er der nogen herinde der har erfaring med at k...
10-14-29,ved ikke lige hvordan disse er i strrelsen
10-16-39,dog kan jeg godt lide pang farver
10-17-297,pengene bliver dog ofte frst udbetalt efter da...


In [22]:
# nlp pipeline for target sentence
base = nlp(sentence_df_cp_op2.loc['7-21-440',:].values[0])

In [23]:
# custom function for similarity scoring between 2 sentences
def calculate_similarity(text2):
    compare = nlp(text2)
    return base.similarity(compare)

In [24]:
# apply on each row
sentence_df_cp_op2['sim_score'] = sentence_df_cp_op2['text'].apply(calculate_similarity)

D:\Anaconda\envs\danish_nlp\lib\site-packages\ipykernel_launcher.py:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
# sort descending
sentence_df_cp_op2.sort_values(by = ['sim_score'], ascending = False, inplace = True)

In [31]:
# top 10 similar sentences, index 0 being the sentence itself
top_10 = sentence_df_cp_op2.iloc[:10,:]['text']

In [39]:
top_10[1]

'nar man kommer slbende med vrkende kn og ildrd nse efter timers vandring gr det godt at stte sig til bordet sammen med de andre gster og bliver krset for i en fransk bjerghytte'

In [34]:
top_10[2]

'stadig flere danskere fatter interesse for den kontante sportsgren hvor veltrnede mnd pa langt over kilo kaster sig efter hinanden'

In [35]:
top_10[3]

'det tager i hvert fald en hel dag at overvre indmarchen af de snorlige rkker af musketerer farverige flagkastere grnne skytter krigere med armbrster byens skyttedronning og de forskellige madvogne som helt bogstaveligt smider brd og plser i hovedet pa publikum'

In [36]:
top_10[4]

'og nok stter to unge mnd sig pa fortovscafeen og far en kop kaffe og taler arabisk men det er cafe latte de taler ogsa dansk og de har parkeret en stor kassevogn foran cafeen'

In [37]:
top_10[5]

'hver lrdag klokken mdes en gruppe kinesiske forldre og deres brn i little mermaid chinese culture school der startede som et privat initiativ i mens forldrene sludrer og dyrker tai chi far brnene undervisning i kinesisk sprog og kultur i de lante klasselokaler'

## Option 3: BERT embedding for Danish > Similarity

In [8]:
# BERT based sentence embedding for Danish
from danish_bert_embeddings import DanishBertEmbeddings
embedder = DanishBertEmbeddings()

In [10]:
# sample embedding
embedding = embedder.embed(target_sentence.values[0])

In [12]:
embedding.shape

torch.Size([768])

In [15]:
# convert to embedding for each sentence
sentence_df_embed = sentence_df['text'].apply(embedder.embed)

In [27]:
# convert to np array
sentence_embed_list = [t.numpy() for t in sentence_df_embed]

### Cosine similarity

In [28]:
# pairewwise cosine similarity between sentence embeddings
cosine_matrix = cosine_similarity(sentence_embed_list)

In [34]:
# convert to dataframe
cosine_df = pd.DataFrame(cosine_matrix, columns = sentence_df['id'], index = sentence_df['id'])

In [52]:
# select the row for target sentence
target_cosine_array = cosine_df.loc['7-21-440',:]
# sort descending
target_cosine_array.sort_values(ascending = False)[:10]

id
7-21-440      1.000000
38-21-615     0.713109
36-0-2615     0.705893
6-37-308      0.705858
51-3-3235     0.699858
36-7-877      0.694900
40-44-510     0.693219
3-28-67       0.692790
38-59-1673    0.692642
51-34-3098    0.692097
Name: 7-21-440, dtype: float32

In [53]:
# gather indices for top 10 similr sentences
top_10_similarity = target_cosine_array.sort_values(ascending = False)[:10].index.tolist()

In [61]:
sentence_df.loc[sentence_df.id == top_10_similarity[1],'text'].values[0]

'Han er fossilekspert, og foruden at være museumsinspektør ved Geomuseum Faxe er han forsker ved Københavns Universitet:" Over revet var der 200- 400 meter havvand, og Thoracosaurus har- akkurat som nulevende havkrokodiller- jagtet sit bytte i de øvre vandlag.'

In [62]:
sentence_df.loc[sentence_df.id == top_10_similarity[2],'text'].values[0]

'På 24 timer mellem 15. og 16. april 1949 fløj 1. 398 maskiner i alt 12. 849 tons fragt ind til den isolerede storby'

In [63]:
sentence_df.loc[sentence_df.id == top_10_similarity[3],'text'].values[0]

'Hovedårsagen til, at Katrina blev så dyr, at skaderne blev så omfangsrige, er ganske enkelt, at der i dag bor langt flere mennesker i kystområderne end tidligere.'

In [64]:
sentence_df.loc[sentence_df.id == top_10_similarity[4],'text'].values[0]

'Men indtil nu er alle teorierne kommet mere eller mindre til kort, da man hele tiden kan finde dyr, der har noget, der ligner menneskelige egenskaber.'

In [65]:
sentence_df.loc[sentence_df.id == top_10_similarity[5],'text'].values[0]

'Børshandlere er såvidt vides udstyret med samme biologiske profil som alle andre mennesker.'

## Option 4: Sub word tokenization > TF-IDF vercorization > cosine similarity

In [6]:
import sentencepiece as spm

In [7]:
# file for subword tokenization training
sub_word_file = '../data/sub_word_training.txt'

In [8]:
all_sentences = '\n'.join(sentence_df['text'])
with open(sub_word_file, 'w') as _file:
    _file.write(all_sentences)

In [9]:
# train sentencepiece model from sub_word_file and makes `m.model` and `m.vocab`
# `m.vocab` is just a reference. not used in the segmentation.
spm.SentencePieceTrainer.train('--input=../data/sub_word_training.txt --model_prefix=m --vocab_size=10000')

In [10]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [11]:
# encode: text => id
print(sp.encode_as_pieces(target_sentence.values[0]))
print(sp.encode_as_ids(target_sentence.values[0]))

['▁If', 'ø', 'lge', '▁Dansk', '▁Kenne', 'l', 'klub', '▁angriber', '▁mu', 'skel', 'hund', 'e', '▁dyr', '▁og', '▁mennesker', '▁cirk', 'a', '▁hver', '▁14', '.', '▁dag', '.']
[930, 0, 184, 363, 2899, 59, 3358, 3042, 4114, 8993, 4280, 16, 1092, 6, 335, 1428, 86, 190, 527, 5, 155, 5]


### Cosine similarity on TF-IDF vectorizer

In [12]:
# create a replica of the sentence dataframe for processing
sentence_df_cp_op4 = sentence_df.copy()
sentence_df_cp_op4.shape

(5000, 2)

In [15]:
# fit count vectorizer with Danish stop words
# use the custom tokenizer with Danish Stemmer
tfidf_vect_sw = TfidfVectorizer(tokenizer=sp.encode_as_ids)#, stop_words=stopwords.words('danish'))
# fit on cleaned text
tfidf_vect_matrix_sw = tfidf_vect_sw.fit_transform(sentence_df_cp_op4.text)
# vocab sze 13574
tfidf_vect_matrix_sw.shape

(5000, 8095)

In [16]:
# find cosine similarity for the count vector matrix
cosine_matrix_ti_sw = cosine_similarity(tfidf_vect_matrix_sw)

In [17]:
# convert to Data Frame
cosine_df_ti_sw = pd.DataFrame(cosine_matrix_ti_sw, columns = sentence_df['id'], index = sentence_df['id'])

In [18]:
# select the row for target sentence
target_cosine_array = cosine_df_ti_sw.loc['7-21-440',:]
# sort descending
target_cosine_array.sort_values(ascending = False)[:10]

id
7-21-440      1.000000
34-7-768      0.186515
47-2-28       0.175104
40-81-2158    0.170774
38-78-1469    0.169889
34-60-944     0.167158
38-20-1059    0.163503
51-33-1121    0.163082
6-37-308      0.154364
36-90-2811    0.149406
Name: 7-21-440, dtype: float64

In [19]:
# gather indices for top 10 similar sentences
top_10_similarity = target_cosine_array.sort_values(ascending = False)[:10].index.tolist()

In [20]:
sentence_df.loc[sentence_df.id == top_10_similarity[1],'text'].values[0]

'Hvor forbrugernes valgmuligheder for 20 år siden kunne tælles på en hånd eller to, så kan de i dag vælge mellem cirka 150 forskellige realkreditlån.'

In [21]:
sentence_df.loc[sentence_df.id == top_10_similarity[2],'text'].values[0]

'I dag'

In [22]:
sentence_df.loc[sentence_df.id == top_10_similarity[3],'text'].values[0]

'Ifølge Henning Otte Hansen må forbrugerne herhjemme nu nok under alle omstændigheder affinde sig med, at maden er relativt dyr:'

In [23]:
sentence_df.loc[sentence_df.id == top_10_similarity[4],'text'].values[0]

'Men det fik ikke den farlige angriber til at skåne City.'

In [24]:
sentence_df.loc[sentence_df.id == top_10_similarity[5],'text'].values[0]

'Ifølge kilderne er brødrene rejst udenlands.'

## Option 5: Sub word tokenization > TOP2VEC for topic clustering

In [25]:
from top2vec import Top2Vec

D:\Anaconda\envs\danish_nlp\lib\site-packages\umap\__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")


In [35]:
topic_model = Top2Vec(list(sentence_df_cp_op4.text),embedding_model='universal-sentence-encoder',speed="deep-learn")

ImportError: universal-sentence-encoder-multilingual is not available.

Try: pip install top2vec[sentence_encoders]

Alternatively try: pip install tensorflow tensorflow_hub tensorflow_text

In [30]:
topic_model.get_num_topics()

2

In [33]:
topic_sizes, topic_nums = topic_model.get_topic_sizes()
topic_sizes, topic_nums

(array([4118,  882], dtype=int64), array([0, 1], dtype=int64))